In [7]:
import pandas
import numpy
import os as os
import re

pandas.set_option('display.max_columns', 500)
pandas.set_option('display.max_rows', 8000000)

### V1

In [8]:
#Read in the combined data file produced by the appender file in the correct format file.
df = pandas.read_csv("09-20NHL.csv")

#Assign column names to the read in files
columns = ["Unnamed: 0","gameDetails", "date","awayBox", "homeBox", "awayAdvanced", "blank5", "blank6","blank7","blank8",
           "blank9","blank10","homeAdvanced","blank12","blank13","blank14","blank15", "blank16","blank17"]

# #set column names and drop unused columns
df.set_axis(columns, axis=1, inplace=True)
df.drop(columns = ["Unnamed: 0",'blank5',"blank6","blank7","blank8","blank9","blank10","blank12",
                   "blank13","blank14","blank15", "blank16","blank17"], inplace= True)

df.head(10)

#use regex to break apart the text recorded data into individual stats
#matches and extracts the away and home teams goals, assists, pts, penalty minutes, even goals, power play goals, 
#short handed goals, shots on goal, and shot on goal percentage
tempDf1 = df['awayBox'].str.extract(r'(^[a-zA-Z]+\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,}\.\d{1,}))', expand = True)
tempDf2 = df['homeBox'].str.extract(r'(^[a-zA-Z]+\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,}\.\d{1,}))', expand = True)
#tempDf2[40:55]
#matches and extracts the away and home teams advanced stats including SATF, SATA, Corsi%, offensive zone start %, hits, blocks, 
tempDf3 = df['awayAdvanced'].str.extract(r'(^[a-zA-Z]+\s(\d{1,})\s(\d{1,})\s(\d{1,}\.\d{1,})\s(\d{1,}\.\d{1,})\s(\d{1,})\s(\d{1,}))', expand = True)
tempDf4 = df['homeAdvanced'].str.extract(r'(^[a-zA-Z]+\s(\d{1,})\s(\d{1,})\s(\d{1,}\.\d{1,})\s(\d{1,}\.\d{1,})\s(\d{1,})\s(\d{1,}))', expand = True)

#matches and extracts the game data, teams names, and team scores
tempDf5 = df['gameDetails'].str.extract(r'^(\d{4}\-\d{1,}\-\d{1,})\s(\w*\.?\w*\s\w*\s?\w*?\s?\w*?)\s(\d{1,2})\s(\w*\.?\w*\s\w*\s?\w*?\s?\w*?)\s(\d{1,2})\s(O?T?).*$')

#assign the parsed data to individual columns
df['AwayGoals'] = tempDf1[1].values
df['AwayAssists'] = tempDf1[2].values
df['AwayPts'] = tempDf1[3].values
df['AwayPim'] = tempDf1[4].values
df['AwayEvg'] = tempDf1[5].values
df['AwayPpg'] = tempDf1[6].values
df['AwayShg'] = tempDf1[7].values
df['AwaySog'] = tempDf1[8].values
df['AwaySogp'] = tempDf1[9].values

df['HomeGoals'] = tempDf2[1].values
df['HomeAssists'] = tempDf2[2].values
df['HomePts'] = tempDf2[3].values
df['HomePim'] = tempDf2[4].values
df['HomeEvg'] = tempDf2[5].values
df['HomePpg'] = tempDf2[6].values
df['HomeShg'] = tempDf2[7].values
df['HomeSog'] = tempDf2[8].values
df['HomeSogp'] = tempDf2[9].values

df['AwaySatf'] = tempDf3[1].values
df['AwaySata'] = tempDf3[2].values
df['AwayCfp'] = tempDf3[3].values
df['AwayOzs'] = tempDf3[4].values
df['AwayHit'] = tempDf3[5].values
df['AwayBlk'] = tempDf3[6].values

df['HomeSatf'] = tempDf4[1].values
df['HomeSata'] = tempDf4[2].values
df['HomeCfp'] = tempDf4[3].values
df['HomeOzs'] = tempDf4[4].values
df['HomeHit'] = tempDf4[5].values
df['HomeBlk'] = tempDf4[6].values

df['Date'] = tempDf5[0]
df['Away'] = tempDf5[1]
df['AwayScore'] = tempDf5[2]
df['Home'] = tempDf5[3]
df['HomeScore'] = tempDf5[4]
df['OT'] = tempDf5[5]

#drop formerly unprocessed data
df.drop(columns = ["gameDetails", "date","awayBox", "homeBox", "awayAdvanced", "homeAdvanced"], inplace= True)

#one hot encoded an overtime 
df['OT'] = numpy.where((df['OT'] == 'OT') | (df['OT'] == 'SO') , 1, 0)

df.dropna(inplace = True)

df = df.astype({
    'AwayGoals': int,
    'AwayAssists': int,
    'AwayPts': int,
    'AwayPim': int,
    'AwayEvg':int,
    'AwayPpg': int,
    'AwayShg': int,
    'AwaySog': int,
    'AwaySogp': float,
    
    'HomeGoals': int,
    'HomeAssists': int,
    'HomePts': int,
    'HomePim': int,
    'HomeEvg': int,
    'HomePpg': int,
    'HomeShg': int,
    'HomeSog': int,
    'HomeSogp': float,
    
    'AwaySatf': int,
    'AwaySata': int,
    'AwayCfp': float,
    'AwayOzs': float,
    'AwayHit': int,
    'AwayBlk': int,
    
    'HomeSatf': int,
    'HomeSata': int,
    'HomeCfp': float,
    'HomeOzs': float,
    'HomeHit': int,
    'HomeBlk': int,
    
    'AwayScore': int,
    'HomeScore': int,
    'OT': int
})
df['AwayGoals'] = df['AwayScore']
df['HomeGoals'] = df['HomeScore']


df['AwayGoalsAllowed'] = df['HomeScore']
df['AwayAssistsAllowed'] = df['HomeAssists']
df['AwayPtsAllowed'] = df['HomePts']
df['AwayEvgAllowed'] = df['HomeEvg']
df['AwayPpgAllowed'] = df['HomePpg']
df['AwayShgAllowed'] = df['HomeShg']
df['AwaySogAllowed'] = df['HomeSog']
df['AwaySogpAllowed'] = df['HomeSogp']

df['HomeGoalsAllowed'] = df['AwayScore']
df['HomeAssistsAllowed'] = df['AwayAssists']
df['HomePtsAllowed'] = df['AwayPts']
df['HomeEvgAllowed'] = df['AwayEvg']
df['HomePpgAllowed'] = df['AwayPpg']
df['HomeShgAllowed'] = df['AwayShg']
df['HomeSogAllowed'] = df['AwaySog']
df['HomeSogpAllowed'] = df['AwaySogp']


columns2 = df.columns.values.tolist()
print(columns2)

# # #prepare for writing of the preprocessed data for offset script

global df1

df1 = df[[
'Date', 'Away', 'AwayScore', 'Home', 'HomeScore', 'OT', 
'AwayGoals', 'AwayAssists', 'AwayPts', 'AwayPim', 'AwayEvg', 'AwayPpg', 'AwayShg', 'AwaySog', 'AwaySogp', 
'HomeGoals', 'HomeAssists', 'HomePts', 'HomePim', 'HomeEvg', 'HomePpg', 'HomeShg', 'HomeSog', 'HomeSogp', 
'AwaySatf', 'AwaySata', 'AwayCfp', 'AwayOzs', 'AwayHit', 'AwayBlk', 
'HomeSatf', 'HomeSata', 'HomeCfp', 'HomeOzs', 'HomeHit', 'HomeBlk', 
          
'AwayGoalsAllowed', 'AwayAssistsAllowed', 'AwayPtsAllowed', 
'AwayEvgAllowed', 'AwayPpgAllowed', 'AwayShgAllowed', 'AwaySogAllowed', 'AwaySogpAllowed', 
          
'HomeGoalsAllowed', 'HomeAssistsAllowed', 'HomePtsAllowed', 
'HomeEvgAllowed', 'HomePpgAllowed', 'HomeShgAllowed', 'HomeSogAllowed', 'HomeSogpAllowed'
]]

print(df1.dtypes)
#df1.describe()

df1.to_excel('Chel_data_proc1.xls')



['AwayGoals', 'AwayAssists', 'AwayPts', 'AwayPim', 'AwayEvg', 'AwayPpg', 'AwayShg', 'AwaySog', 'AwaySogp', 'HomeGoals', 'HomeAssists', 'HomePts', 'HomePim', 'HomeEvg', 'HomePpg', 'HomeShg', 'HomeSog', 'HomeSogp', 'AwaySatf', 'AwaySata', 'AwayCfp', 'AwayOzs', 'AwayHit', 'AwayBlk', 'HomeSatf', 'HomeSata', 'HomeCfp', 'HomeOzs', 'HomeHit', 'HomeBlk', 'Date', 'Away', 'AwayScore', 'Home', 'HomeScore', 'OT', 'AwayGoalsAllowed', 'AwayAssistsAllowed', 'AwayPtsAllowed', 'AwayEvgAllowed', 'AwayPpgAllowed', 'AwayShgAllowed', 'AwaySogAllowed', 'AwaySogpAllowed', 'HomeGoalsAllowed', 'HomeAssistsAllowed', 'HomePtsAllowed', 'HomeEvgAllowed', 'HomePpgAllowed', 'HomeShgAllowed', 'HomeSogAllowed', 'HomeSogpAllowed']
Date                   object
Away                   object
AwayScore               int32
Home                   object
HomeScore               int32
OT                      int32
AwayGoals               int32
AwayAssists             int32
AwayPts                 int32
AwayPim                